### Import

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models, optimizers
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import sqlite3
import glob
import csv
pd.set_option('display.max_row', 40000)
pd.set_option('display.max_column', 10000)
%matplotlib inline
from os.path import join
# tf 2.0부터 keras는 tensorflow의 공식 API


### Hyper Parameters

In [2]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

tf.random.set_seed(777)

### Creating a Checkpoint Directory

In [3]:
cur_dir = os.getcwd()
ckpt_dir_name = 'checkpoints'
model_dir_name = 'minst_cnn_seq'

checkpoint_dir = os.path.join(cur_dir, ckpt_dir_name, model_dir_name)
os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint_prefix = os.path.join(checkpoint_dir, model_dir_name)

### Data

In [4]:
path = "src/*" #-------------------------자신의 db파일이 있는 폴더로 수정---------------------------
file_list =  glob.glob(path)
file_list_db = [file for file in file_list if file.endswith(".db")]
print ("file_list: {}".format(file_list_db))
len(file_list_db)
# EventCode03이 일어났을 당시의 초당 주행기록 가져오기
srcrec_df2 = pd.DataFrame() #연결한 db 결과 저장소
for i in range (0,len(file_list_db)):
    # event 파일 가져오기
    f = open('src/event.csv') #-------------------------자신의 event파일이 있는 폴더로 수정---------------------------
    csvReader = csv.reader(f)
    #db연결
    conn = sqlite3.connect(file_list_db[i])
    c = conn. cursor()
    # event 테이블 유무 확인 후, 있으면 제거
    c.execute('Drop Table If Exists event')
    # event 테이블 생성
    c.execute("create table event(CAR_RECDRV_KEY integer, EVENT_CODE text, EVENT_STDT text, EVENT_ENDT text)")
    # csv 파일 읽어 온 데이터 insert
    for row in csvReader:
        if row[7] == "EVENT_CODE":
            continue
        sql1 = "insert into event (CAR_RECDRV_KEY,EVENT_CODE, EVENT_STDT, EVENT_ENDT) values (?,?,?,?)"
        key= int(row[1])
        #key 추출
        if key != int(file_list_db[i][4:11]): #src에 자신의 db파일이 있어야함, 아니면 인덱스 수정할 것
            continue
        code = (row[7])
        #event 추출
        if code[-11:] != "EventCode03":
            continue
        stdt = (row[8])
        endt = (row[9])
        c.execute(sql1,(key,code,stdt,endt))
    #트랜잭션 저장
    conn.commit()
    #event03 뽐기 query 실행
    sql2 = 'SELECT SRCREC.srcValue, SRCREC.realTime, SRCREC.srcSpeed, SRCREC.srcAPS,\
    SRCREC.srcGyroValue, SRCREC.srcRPM, SRCREC.srcTPS, SRCREC.srcMAF, SRCREC.srcEngineLoad, ifnull(EVENT_CODE, "0") FROM SRCREC LEFT OUTER JOIN\
    (SELECT EVENT_CODE, EVENT_STDT s, EVENT_ENDT e FROM event\
    ) ON SRCREC.realTime BETWEEN strftime("%Y%m%d%H%M%S",s,"-6 seconds")\
     AND strftime("%Y%m%d%H%M%S",s,"-1 seconds")'
    query =  c.execute(sql2)
    cols = [column[0] for column in query.description]
    srcrec_df = pd.DataFrame.from_records(data=query.fetchall(), columns=cols)
    #db 파일 연결 작업
    srcrec_df2 = pd.concat([srcrec_df2, srcrec_df], ignore_index=True)
    #db 연결 종료
    c.close()
    conn.close()
    #파일 연결 종료
    f.close()
srcrec_df = srcrec_df2

file_list: ['src\\3048682.db']


In [5]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

tf.random.set_seed(777)  # for reproducibility

In [6]:
y_data = srcrec_df.values[:,9]
y_data = pd.get_dummies(y_data).values

y_data = y_data[:,0]

where_0 = np.where(y_data ==0)
where_1 = np.where(y_data ==1)

y_data[where_0] = 1
y_data[where_1] = 0
y_data = to_categorical(y_data)

y_data

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [7]:
x_data = srcrec_df.apply(pd.to_numeric,errors='coerce')


x_data = x_data.iloc[:,2:9]
x_data = x_data.values
x_data= x_data.reshape(len(x_data),1,-1)
x_data

array([[[ 4.        , 14.90196   ,  5.64022684, ..., 14.117647  ,
          0.        ,  0.        ]],

       [[ 3.        , 14.90196   ,  6.97546625, ..., 14.117647  ,
          0.        , 49.80392   ]],

       [[ 3.        , 14.90196   ,  5.53535843, ..., 14.117647  ,
          0.        , 49.80392   ]],

       ...,

       [[ 0.        , 14.90196   , -0.08611873, ..., 12.156863  ,
          0.        , 38.039215  ]],

       [[ 0.        , 14.90196   ,  0.48863536, ..., 12.156863  ,
          0.        , 38.039215  ]],

       [[ 0.        , 14.90196   ,  0.28494966, ..., 12.156863  ,
          0.        , 39.215687  ]]])

In [8]:
from sklearn.model_selection import train_test_split
trnx, tstx, trny, tsty = train_test_split(x_data, y_data, test_size=0.3, random_state=111)
trnx = np.expand_dims(trnx,axis=-1)
tstx = np.expand_dims(tstx,axis=-1)
print(len(trnx))
print(len(tstx))
print(len(trny))

1638
703
1638


### Datasets

In [9]:
train_dataset = tf.data.Dataset.from_tensor_slices((trnx, trny)).shuffle(
                buffer_size=15000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((tstx, tsty)).batch(batch_size)

In [10]:
train_dataset

<BatchDataset shapes: ((None, 1, 7, 1), (None, 2)), types: (tf.float64, tf.float32)>

In [11]:
test_dataset

<BatchDataset shapes: ((None, 1, 7, 1), (None, 2)), types: (tf.float64, tf.float32)>

### Model Function

In [12]:
def create_model():
    model = keras.Sequential()#이제 계층을 순차적으로 연결 가능
    model.add(keras.layers.Conv2D(filters=32, kernel_size=3, activation=tf.nn.relu, padding='SAME', 
                                  input_shape=(1, 7, 1)))# input_shape => 첫 layer에만 선언
    model.add(keras.layers.MaxPool2D(padding='SAME'))
    model.add(keras.layers.Conv2D(filters=64, kernel_size=3, activation=tf.nn.relu, padding='SAME'))
    model.add(keras.layers.MaxPool2D(padding='SAME'))
    model.add(keras.layers.Conv2D(filters=128, kernel_size=3, activation=tf.nn.relu, padding='SAME'))
    model.add(keras.layers.MaxPool2D(padding='SAME'))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(256, activation=tf.nn.relu))
    model.add(keras.layers.Dropout(0.4))
    model.add(keras.layers.Dense(2))
    return model

In [13]:
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 1, 7, 32)          320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 1, 4, 32)          0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 1, 4, 64)          18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 1, 2, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 2, 128)         73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 1, 1, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 128)               0

### Loss Function

In [14]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.binary_crossentropy(
        y_pred=logits, y_true=labels, from_logits=True))    
    return loss

### Calculating Gradient

In [15]:
def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

### Caculating Model's Accuracy

In [16]:
def evaluate(model, images, labels):
    logits = model(images, training=False)
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return accuracy

### Optimizer

In [17]:
# adam 사용
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

### Creating a Checkpoint

In [18]:
checkpoint = tf.train.Checkpoint(cnn=model)

### Training

In [19]:
def train(model, images, labels):
    grads = grad(model, images, labels)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_loss = 0.
    avg_train_acc = 0.
    avg_test_acc = 0.
    train_step = 0
    test_step = 0    
    
    for images, labels in train_dataset:
        train(model, images, labels)
        #grads = grad(model, images, labels)                
        #optimizer.apply_gradients(zip(grads, model.variables))
        loss = loss_fn(model, images, labels)
        acc = evaluate(model, images, labels)
        avg_loss = avg_loss + loss
        avg_train_acc = avg_train_acc + acc
        train_step += 1
    avg_loss = avg_loss / train_step
    avg_train_acc = avg_train_acc / train_step
    
    for images, labels in test_dataset:        
        acc = evaluate(model, images, labels)  
        avg_test_acc = avg_test_acc + acc
        test_step += 1    
    avg_test_acc = avg_test_acc / test_step    

    print('Epoch:', '{}'.format(epoch + 1), 'loss =', '{:.8f}'.format(avg_loss), 
          'train accuracy = ', '{:.4f}'.format(avg_train_acc), 
          'test accuracy = ', '{:.4f}'.format(avg_test_acc))
    
    checkpoint.save(file_prefix=checkpoint_prefix)

print('Learning Finished!')

Learning started. It takes sometime.
Epoch: 1 loss = 0.24861465 train accuracy =  0.9742 test accuracy =  0.9672
Epoch: 2 loss = 0.15915801 train accuracy =  0.9742 test accuracy =  0.9672
Epoch: 3 loss = 0.13522723 train accuracy =  0.9740 test accuracy =  0.9672
Epoch: 4 loss = 0.12047034 train accuracy =  0.9742 test accuracy =  0.9672
Epoch: 5 loss = 0.11309789 train accuracy =  0.9742 test accuracy =  0.9672
Epoch: 6 loss = 0.10665134 train accuracy =  0.9740 test accuracy =  0.9672
Epoch: 7 loss = 0.10429364 train accuracy =  0.9742 test accuracy =  0.9672
Epoch: 8 loss = 0.09898535 train accuracy =  0.9741 test accuracy =  0.9672
Epoch: 9 loss = 0.09745111 train accuracy =  0.9740 test accuracy =  0.9672
Epoch: 10 loss = 0.09397714 train accuracy =  0.9744 test accuracy =  0.9672
Epoch: 11 loss = 0.09688950 train accuracy =  0.9741 test accuracy =  0.9679
Epoch: 12 loss = 0.09946250 train accuracy =  0.9745 test accuracy =  0.9668
Epoch: 13 loss = 0.09801816 train accuracy =  0.

### loss: 0.01  test Accuracy : 99.67% 